In [1]:
# Constrained water and energy balance estimation 
# v4.2: adapted from v4.1 by treating SCF known, G estimated offline
# v4.3: adapted from v4.2 & 3.3 to handle to batch processing on OSC
# v4.4: adapted from v4.3 to update ipopt deprecation
# v4.5: moving σWRFG based on SCF 7/2/20 JLD
# v4.6: adding SDC to objective function: MD & JD: 9/28/20
# v4.7: adding "pseudo-valid prior" to starting points. remove SDC. MD: 1/7/21
# v4.8: tweak error parameters. MD & JD: 1/8/21
# v4.9 prior valid added to objective function. daily G capped in constraints. more outputs written 10/5/21
# v4.9.1 remove SWE from objective
# v4.9.2 added air temp
# v51 Converted to function but still uses textfile as input
# v52 Prototyping to pass the array directly to function [and fixed tab to 4 spaces]
# v53 To set tab to 4 spaces (by copying/pasting)
# v54 combine 9 separate textfile output in one txt file (due to file count limitation on Discover)
# v55 Fixing error due to missing days of data due to Polar nights
# v58 New updates by jack (Nov 2023)
# Jan 29, 2024: Due to error on obj function σWRFG becoming zero, fixed the minimum σWRFG to 25 
# Jun 21, 2024: No change for 1km run here. MODIS with UINT8 did not work likely because of missing due to no-data background and polar nights.
# Oct 06, 2025: Updated main Blender function to handle new precip scaling algorithm.


using NCDatasets
using Random
using JuMP
using Ipopt
using DelimitedFiles
using Statistics
using Rasters
using Distributed
Random.seed!(1234)  # seed for reproducibility


TaskLocalRNG()

In [11]:
function blender(i, j, SWEprior, Pprior, Gprior, SCFinst, AirT, PSval, logDir, exp_dir, twindow=2)
  """
  changing twindow from 2 to 1 (May 19, 2025)
  inputs
  ==============
  i,j are pixel locations
  SWEprior, Pprior, Gprior are prior estimates of SWE [m], Precipitation [m] and surface heat flux [W/m2]
  SCFinst is measured snow covered fraction
  AirT is air temperature [K]
  logdir and exp_dir are directories where log and data are written
    
  variable sizes
  =============== 
  SWE and SCF are storage terms, so will be length nt. 
  P, Melt, and G are flux terms, so will be length nt-1
  for input, all variables are length nt, so just use Pprior[1:nt-1]. Note, Gprior currently unused
  for output in section 4, will output fluxes as nt, with the last element set to 0.    

  """
    # 0 handle variable sizes
    nt = length(SCFinst)
    Pprior = Pprior[1:nt-1]

    # TODO This is prototype only to Fill in missing SCF values
    #  Write a function that returns SCF based on LIS SWE prior.
    for i=1:nt
        if ismissing(SCFinst[i])
            if SWEprior[i] <= 0  # if SWE is zero, then SCF is zero
                SCFinst[i] = 0
            elseif SWEprior[i] > 0 && SWEprior[i] < 0.1
                SCFinst[i] = 0.1
            elseif SWEprior[i] >= 0.1 && SWEprior[i] < 0.5
                SCFinst[i] = 0.5
            elseif SWEprior[i] >= 0.5
                SCFinst[i] = 0.75
            end
        end
    end

    # 1 Smooth SCF observations
    # twindow = 5
    SCFobs = smoothdata(SCFinst, twindow, nt, "median")
    SCF_smooth_season = smoothdata(SCFinst, 30, nt, "mean")  # before it was 60
    # Quick check to make sure scalar never nans out
    Pprior = PSval.*Pprior;
    SWEprior = PSval.*SWEprior;

    # 2 Define hyperparameters
    tmelt,tmelt_smooth,SWEmax,SWEmin_global,Meltmax,σP,σSWE,k,Melt0,L=define_hyperparameters(SCF_smooth_season, nt, Pprior, SWEprior, AirT, SCFobs)

    tol = 0.0      # or a small tolerance like 1e-12 if your var1 has noise
    eps = 1e-9     # to approximate strict ">" with "≥ eps"
    
    # 3 Solve
    m = Model(optimizer_with_attributes(Ipopt.Optimizer,"max_iter"=>5000))
    # set_silent(m)
    # define variables and bounds
    @variable(m, SWEmin_global <= SWE[i=1:nt] <= SWEmax[i], start=SWEprior[i])
    @variable(m, Precip[i=1:nt-1]>=0. ,start=Pprior[i]);
    @variable(m, 0. <= Melt[i=1:nt-1]<= Meltmax)
    @variable(m, Mcost[i=1:nt-1] >=0)
    #@expression(m, penaltySWE[i=1:nt-1], (1-tmelt[i])*1*(SWEprior[i]-SWE[i]))

    # define constraints
    for i in 1:nt-1
      @NLconstraint(m,Mcost[i]==L/(1+exp( -k*(Melt[i]-Melt0))))
    end
    for i in 1:nt-1
      @constraint(m,SWE[i+1]==SWE[i]+Precip[i]-Melt[i])
    end
    # define objective function
    @objective(m,Min,sum((Precip-Pprior).^2 ./σP.^2) + sum((SWE-SWEprior ).^2 ./ σSWE.^2) + sum(Mcost.^2) )#+ sum(penaltySWE))
    log_file =  "$logDir/Pix_$(i)_$(j).txt"  # _$(twindow)
    # solve
    redirect_stdio(stdout=log_file, stderr=log_file) do
      optimize!(m)
    end
    # print(termination_status(m))
    
    # 4 extract
    NODATAvalue = -9999
    SWEhat=JuMP.value.(SWE)
    Phat=zeros(nt,1)
    Phat[1:nt-1] = JuMP.value.(Precip)
    Melt_hat = zeros(nt,1)
    Melt_hat[1:nt-1] = JuMP.value.(Melt);
    
    Δt = 86400; #s/day
    ρw = 1000; #density of water
    Lf = 0.334E6; #Latent heat of fusion J/kg    
    GmeltHat = Melt_hat/Δt*Lf*ρw
    
    Ghat = ones(nt,1)*NODATAvalue
    Ushat = ones(nt,1)*NODATAvalue
    G_pv = ones(nt,1)*NODATAvalue
    U_pv = ones(nt,1)*NODATAvalue
    Gmelt_pv = ones(nt,1)*NODATAvalue
    SWEpv = ones(nt,1)*NODATAvalue
    
    # 5 output
    out_vars = hcat(SWEhat, GmeltHat, Ghat, Phat, Ushat, G_pv, Gmelt_pv, U_pv, SWEpv, SCFobs)
    writedlm("$(exp_dir)/Pix_$(i)_$(j).txt", out_vars)
    # writedlm("$(exp_dir)/SWE.txt", SWEhat)
    # writedlm("$(exp_dir)/Phat.txt", Phat)
    # writedlm("$(exp_dir)/SCFhat.txt", SCFobs)
    # writedlm("$(exp_dir)/Meltmax.txt", Meltmax)
    # # 6 clean up
    m = nothing
    GC.gc()
    return nothing    
end


function smoothdata(SCF_inst, twindow, nt, smoothfunc)
    """
    Smooth data using a moving average or median filter.
    Parameters:
    ============
    SCF_inst: Input data to be smoothed.
    twindow: Window size for smoothing.
        example: 1 means smooth using 1 data point on either side of the current point.
        In this formulation, use half the value that was originally used by Mike (ie, 60 is now 30 etc.).
    nt: Length of the input data.
    smoothfunc: Type of smoothing function ('mean' or 'median').

    """
    # println("Inside Smoothing data...")
    # println("$twindow, $nt, $smoothfunc")
    # better to copy "SCF_inst" so the smooth function doesn't modify the original data. Even better apppend border on both sides of the data.
    SCF_smooth=zeros(nt,1)
    for i=(1+twindow):(nt-twindow)
        # adding 1 (ie, 1+twindow) in the for loop because Julia is 1-based indexing
        if smoothfunc == "mean"
            SCF_smooth[i] = mean(SCF_inst[i-twindow:i+twindow])
        elseif smoothfunc=="median"
            SCF_smooth[i] = mean(SCF_inst[i-twindow:i+twindow])
        end
    end    
    return SCF_smooth
end

function define_uncertainty(Pprior, SWEprior, AirT, SCFobs, nt, tmelt_smooth)
    # convert air temperature K-> C
    AirT = AirT.-273.15
    
    # 2.2.1 Precipitation Uncertainty
    RelPUnc = 0.3; #[-] this applies to cumulative precipitation
    # Uncertainty for accumulation . precip is size nt-1
    σP = zeros(nt-1,1)
    σPmin = 0.001
    Pthresh = 0.001
    for i=1:nt-1
      if Pprior[i]<Pthresh
        σP[i]=σPmin
      else
        σP[i]=Pprior[i]*RelPUnc
      end
    end    
    # adjust uncertainty to apply to the number of snow days
    nsnowday = 0
    Tprecip_thresh = 1.5
    for i=1:nt-1
        if Pprior[i]>Pthresh && AirT[i] < Tprecip_thresh
            nsnowday += 1
        end
    end
    if nsnowday > 0
        σP = σP*sqrt(nsnowday);    
    end
    
    # 2.2.2 SWE Uncertainty
    fSWE = 0.4
    σSWE = SWEprior*fSWE;
    σSWEmin = 0.01
    σSWEmax = 10
    for i=1:nt
        # if SWEprior[i]>0 && SCFobs[i]==0
        if SCFobs[i]==0 || tmelt_smooth[i]>.1    
            σSWE[i] = σSWEmax
        end
        if σSWE[i] < σSWEmin
            σSWE[i] = σSWEmin
        end
    end   
    
    return σP, σSWE
end

function define_hyperparameters(SCF_smooth_season,nt,Pprior,SWEprior,AirT, SCFobs)
    """
    list of hyperparameters
    =======================
    twindow for smoothing SCF for snow on/off constraint - defined in main
    twindow for smoothing SCF for identifying melt times - defined in main
    ΔSCFthresh
    twindow for smoothing melt times
    SWEmax_global
    SWEmin_global
    Meltmax    
    k,Melt0,L
    σPmin - defined in define_uncertainty
    Pthresh -  defined in define_uncertainty
    Tprecip_thresh -  defined in define_uncertainty
    fSWE -  defined in define_uncertainty
    σSWEmin -  defined in define_uncertainty
    σSWEmax -  defined in define_uncertainty        
    
    """
    
    # 2.1 Define prior estimates
    # 2.1.1 Define times when snow is melting 
    tmelt = zeros(nt,1)
    ΔSCFthresh = -0.01
    for i=2:nt
        if SCF_smooth_season[i] - SCF_smooth_season[i-1] < ΔSCFthresh
            tmelt[i] = 1
        end
    end    
    # twindow = 30
    tmelt_smooth = smoothdata(tmelt, 15, nt, "mean")  # before it was 30
    
    # 2.2 Extreme / limit values
    # 2.2.1 SWE
    SWEmax_global = 5
    SWEmin_global = 1.0e-6 #1/1000 mm TODO (BNY) we can use this for missing values due flags such as to water etc. ie, give those pixels a zero value
    # define SWEmax as a function of time and of SCF
    #    set SWEmax to 0 if SCF is low
    SWEmax = zeros(nt,1)
    for i = 1:nt
      if SCFobs[i] == 0
        SWEmax[i] = SWEmin_global
      else
        SWEmax[i] = SWEmax_global
      end
    end    
    #2.2.2 Melt
    #Meltmax = define_maxmelt(AirT,SCFobs,nt)
    #Meltmax = ones(nt,1); Meltmax = Meltmax.*0.075
    Meltmax = 0.075
    
    # 2.3 Define uncertainty
    σP,σSWE = define_uncertainty(Pprior, SWEprior, AirT, SCFobs, nt, tmelt_smooth)
    # 2.4 Melt cost function parameters
    k = 500
    Melt0 = 0.05
    L = 1
    
    return tmelt, tmelt_smooth, SWEmax, SWEmin_global, Meltmax, σP, σSWE, k, Melt0, L
end

function define_maxmelt(AirT,SCFobs,nt)
    # convert air temperature K-> C
    AirT = AirT.-273.15
    # Define degree deltas scfDelta meltmax
    dday = zeros(nt,1)
    scfDelta = zeros(nt,1);
    Meltmax = zeros(nt,1);
    for i = 2 : nt
        scfDelta[i] = SCFobs[i] - SCFobs[i];
    end

    for i = 1:nt
        if AirT[i] >= 0 && scfDelta[i] <=0     # temperature is above freezing
            if i == 1
                dday[i] = 1
            else
                dday[i] = dday[i-1] + 1
            end
        else
            dday[i] = 0;   # reset counter when below freezing
        end
    end

    for i = 1 : nt
        Meltmax[i] = dday[i] * 0.005;
    end

    # for i = 1 : nt
    #     if Meltmax[i] > 0.02
    #         Meltmax[i] = 0.02
    #     end
    # end
    return Meltmax
end

define_maxmelt (generic function with 1 method)

In [16]:
function call_Blender_wshed(WY,wshedName)
    fullName = WY  # WY2016
    water_year = fullName[end-1:end] #last 4 chars are assumed year, else error. 
    DataDir = "/Users/jldechow/Documents/Projects/UNC/CoReSSD/fullData/WY$(water_year)"
    wshed = wshedName
    exp_dir = "$DataDir/$wshed/"
    logDir = "$DataDir/logs/"
    mkpath(logDir)
    mkpath(exp_dir)
# 2. Read the Input netCDF file
    files = (
    SCF          = "$DataDir/SCF.nc",             # variable inside: SCF
    # Snowf_tavg   = "$DataDir/Snowf_tavg_gf.nc",   # variable inside: Snowf_tavg smoothing sigma=1
    # SWE_tavg     = "$DataDir/SWE_tavg_gf.nc",     # variable inside: SWE_tavg smoothing sigma=1
    Snowf_tavg   = "$DataDir/Snowf_tavg_gf2.nc",   # variable inside: Snowf_tavg smoothing sigma=0.5
    SWE_tavg     = "$DataDir/SWE_tavg_gf2.nc",     # variable inside: SWE_tavg smoothing sigma=0.5
    # Snowf_tavg   = "$DataDir/Snowf_tavg.nc",   # variable inside: Snowf_tavg
    # SWE_tavg     = "$DataDir/SWE_tavg.nc",     # variable inside: SWE_tavg
    Tair_f_tavg  = "$DataDir/Tair_f_tavg.nc",     # variable inside: Tair_f_tavg
    Qg_tavg      = "$DataDir/Qg_tavg.nc",         # variable inside: Qg_tavg
)

A = RasterStack((
    SCF         = Raster(files.SCF;         name=:SCF,         lazy=true),
    Snowf_tavg  = Raster(files.Snowf_tavg;  name=:Snowf_tavg,  lazy=true),
    SWE_tavg    = Raster(files.SWE_tavg;    name=:SWE_tavg,    lazy=true),
    Tair_f_tavg = Raster(files.Tair_f_tavg; name=:Tair_f_tavg, lazy=true),
    Qg_tavg     = Raster(files.Qg_tavg;     name=:Qg_tavg,     lazy=true),
))
    #A = RasterStack(files; lazy=true)  # lazy=true https://github.com/rafaqz/Rasters.jl/issues/449
    # Read Precip Scalar Files as single raster
    # Time variable is not same as others so can't read into main stack
    PS = Raster("$DataDir/precip_scalar.nc"; name=:precip_scalar, lazy=true);
    #@info("Time until creating Rasterstack from separate input netcdf files = $(round(running_time, digits=2)) minutes")
    
    # Subset only the required variables because the nc file can have extraneous vars that cause problem with julia
    A = A[(:Snowf_tavg, :SWE_tavg, :Tair_f_tavg, :Qg_tavg, :SCF)];  # to remove spatial ref that cause problem during subsetting
    
    
    @info("Watershed Run  ")
    
    if wshed == "RGV"
        #minLat = 36.995; maxLat = 38.005; minLon = -107.57; maxLon = -106.16;
        # x0, x1 = -107.573, -106.33  # longitude
        # y0, y1 = 37.38, 38      # latitude    
        x0, x1 = -107.57, -106.16  # longitude
        y0, y1 = 36.95, 38.005     # latitude    
    end
    
    A = A[X(Between(x0, x1)), Y(Between(y0, y1))]  # Another way of getting the same chip around watershed
    PS = PS[X(Between(x0, x1)), Y(Between(y0, y1))]
    
    valid_pix_ind = findall(!ismissing, A["Qg_tavg"][Ti=1])  # Extract cartesian index for non-missing (ie, all) data using one-day of data 
    valid_pix_count = length(valid_pix_ind)
    ind = valid_pix_ind
    
    # Fix the nt = 1 issue for precip scalar
    nDayFix = size(A[:SWE_tavg], 3) 
    PS_T = ndims(PS) == 2 ? reshape(PS, size(PS,1), size(PS,2), 1) : PS
    PS_T = repeat(PS_T, 1, 1, nDayFix)  # X×Y×nt


    for ij in ind
        i = ij[1]
        j = ij[2]
        WRFSWE = A["SWE_tavg"][X=i, Y=j].data/1000  # Here "data" is an AbstractArray.
        WRFP = A["Snowf_tavg"][X=i, Y=j].data/1000
        WRFG = A["Qg_tavg"][X=i, Y=j].data
        AirT = A["Tair_f_tavg"][X=i, Y=j].data/100
        MSCF = A["SCF"][X=i, Y=j].data; #/100 Convert to fraction. multiplication and devision works without dot. but add/subtract will need dot.
        PSval = PS_T[i,j,1];
        blender(i, j, WRFSWE, WRFP, WRFG, MSCF, AirT, PSval, logDir, exp_dir)
    end
end

call_Blender_wshed (generic function with 1 method)

In [17]:
wshedName = "RGV";
WY = "WY2015"

call_Blender_wshed(WY,wshedName)

[ Info: Watershed Run  
